## XGBoost Model on Diff Data
USA World Series Results,
Run on "Diff" data

# @To Do

- [ ] Look at scaling or normalizing data
- [ ] Look at Neural Networks
- [ ] Resolve issue of target being numeric or string label
- [ ] Explore relationship between Posession Time + Attacking Rucks + Passes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#Import Data - only want USA matches
df = pd.read_csv("../data/output/new_features_diffdata.csv")
df.head()

,Opp,Tournament,Poss_Time_Diff,Score_Diff,Conv_Diff,Tries_Diff,Passes_Diff,Contestable_KO_Win_pct_Diff,PenFK_Against_Diff,RuckMaul_Diff,...,-99 : -75,-74 : -25,-24 : -1,0 : 25,26 : 50,51 : 75,76 : 100,101 : 125,126 : 150,Result
0,AUSTRALIA,2015_Cape_Town,13.966480,-10.638298,-14.285714,0.250000,25.925926,-50.000000,0.000000,0.000000,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,WALES,2015_Cape_Town,7.471264,15.555556,14.285714,0.083333,27.868852,25.000000,-20.000000,-100.000000,...,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,1
2,KENYA,2015_Cape_Town,-33.136095,-44.444444,-33.333333,-0.750000,-10.638298,-16.666667,66.666667,60.000000,...,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,NEW ZEALAND,2015_Cape_Town,51.758794,33.333333,33.333333,0.000000,76.119403,-75.000000,-50.000000,-100.000000,...,-37.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,FIJI,2015_Cape_Town,12.880562,-20.833333,-25.000000,0.266667,38.461538,-66.666667,-33.333333,-33.333333,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
df.info()
list(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 27 columns):
Opp                            156 non-null object
Tournament                     156 non-null object
Poss_Time_Diff                 156 non-null float64
Score_Diff                     156 non-null float64
Conv_Diff                      156 non-null float64
Tries_Diff                     156 non-null float64
Passes_Diff                    156 non-null float64
Contestable_KO_Win_pct_Diff    156 non-null float64
PenFK_Against_Diff             156 non-null float64
RuckMaul_Diff                  156 non-null float64
Ruck_Win_pct_Diff              156 non-null float64
Cards_diff                     156 non-null float64
Lineout_Win_Pct_Diff           156 non-null float64
Scrum_Win_Pct_Diff             156 non-null float64
-175 : -150                    156 non-null float64
-149 : -125                    156 non-null float64
-124 : -100                    156 non-null float64
-99 : -75    

['Opp',
 'Tournament',
 'Poss_Time_Diff',
 'Score_Diff',
 'Conv_Diff',
 'Tries_Diff',
 'Passes_Diff',
 'Contestable_KO_Win_pct_Diff',
 'PenFK_Against_Diff',
 'RuckMaul_Diff',
 'Ruck_Win_pct_Diff',
 'Cards_diff',
 'Lineout_Win_Pct_Diff',
 'Scrum_Win_Pct_Diff',
 '-175 : -150',
 '-149 : -125',
 '-124 : -100',
 '-99 : -75',
 '-74 : -25',
 '-24 : -1',
 '0 : 25',
 '26 : 50',
 '51 : 75',
 '76 : 100',
 '101 : 125',
 '126 : 150',
 'Result']

In [7]:
#Create a list of features to drop that are unneccessary or will bias the prediction
droplist = ['Opp', 'Score_Diff', 'Tries_Diff','Tournament', 'Conv_Diff','-175 : -150', '-149 : -125','-124 : -100', '-99 : -75', '-74 : -25','-24 : -1','0 : 25','26 : 50','51 : 75','76 : 100','101 : 125','126 : 150']

xg_data = df.drop((droplist), axis=1)

#Drop rows with Result == "T" (Ties). This label messes up classification models
xg_data.drop(xg_data[xg_data.Result == 2].index, inplace=True)

#Pull out the variable we're trying to predict: 'Result'
X = xg_data.drop('Result',axis=1)
y = xg_data['Result']

In [9]:
xg_data.head()
#Check to insure 'Result' only contains 2 values (W, L)
xg_data['Result'].describe()
xg_data.describe()

,Poss_Time_Diff,Passes_Diff,Contestable_KO_Win_pct_Diff,PenFK_Against_Diff,RuckMaul_Diff,Ruck_Win_pct_Diff,Cards_diff,Lineout_Win_Pct_Diff,Scrum_Win_Pct_Diff,Result
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,10.402130,17.196314,2.491328,-1.993963,-0.644381,0.062044,5.298013,0.124614,-0.081126,0.562914
std,20.750082,26.935639,45.526240,49.889728,59.542377,0.198595,27.780280,0.668788,0.637047,0.497677
min,-40.000000,-48.837209,-166.666667,-100.000000,-100.000000,-0.550000,-50.000000,-1.000000,-1.000000,0.000000
25%,-3.491112,0.793651,-27.500000,-33.333333,-33.333333,-0.083333,0.000000,0.000000,-0.500000,0.000000
50%,11.166253,16.129032,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,1.000000
75%,23.795151,34.495192,33.333333,33.333333,33.333333,0.181187,0.000000,0.750000,0.000000,1.000000
max,72.272727,90.909091,133.333333,100.000000,100.000000,0.875000,100.000000,1.000000,1.000000,1.000000


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Train an XGBoost Classifier Model (v.2)
Info from https://cambridgespark.com/content/tutorials/hyperparameter-tuning-in-xgboost/index.html

In [12]:
import xgboost as xgb

#Build Dmatrices
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

## Build Baseline Model
Use mean absolute error (MAE) to evaluate the quality of our predictions.

In [13]:
from sklearn.metrics import mean_absolute_error

In [14]:
import numpy as np
# "Learn" the mean from the training data
mean_train = np.mean(y_train)

# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train

# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)

print("Baseline MAE is {:.2f}".format(mae_baseline))

Baseline MAE is 0.50


### The params dictionary
Most of the parameters passed to XGBoost via the native API are defined in a dictionary. Let's define it with default values for the moment. 

In [15]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective': 'binary:logistic',
}

#### Parameters num_boost_round and early_stopping_rounds

In [16]:
params['eval_metric'] = "mae"
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-mae:0.497306
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:0.49786
[2]	Test-mae:0.505776
[3]	Test-mae:0.506648
[4]	Test-mae:0.511049
[5]	Test-mae:0.517777
[6]	Test-mae:0.509068
[7]	Test-mae:0.51442
[8]	Test-mae:0.503136
[9]	Test-mae:0.511003
[10]	Test-mae:0.507403
Stopping. Best iteration:
[0]	Test-mae:0.497306



Stopping. Best iteration:

[30]	Test-mae:0.431734

In [17]:
print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

Best MAE: 0.50 with 1 rounds


### Using XGBoost's CV

In [18]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=101,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

cv_results

,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,0.431809,0.006453,0.482325,0.029471
1,0.382849,0.013520,0.473460,0.029591
2,0.342852,0.014201,0.454317,0.037009
3,0.307023,0.015926,0.444842,0.038862
4,0.277907,0.015661,0.439179,0.038921
5,0.253512,0.014012,0.434898,0.040495
6,0.233697,0.015428,0.424995,0.043650
7,0.217407,0.015237,0.416792,0.051622
8,0.202807,0.013905,0.415034,0.056817
9,0.189673,0.013357,0.409693,0.059733


In [19]:
cv_results['test-mae-mean'].min()

0.3857912

### Parameters max_depth and min_child_weight

In [20]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

In [21]:
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))

    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight

    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=101,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )

    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)

print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=9, min_child_weight=5
	MAE 0.4243068 for 51 rounds
CV with max_depth=9, min_child_weight=6


/Users/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	MAE 0.42898420000000004 for 64 rounds
CV with max_depth=9, min_child_weight=7
	MAE 0.43361279999999996 for 55 rounds
CV with max_depth=10, min_child_weight=5
	MAE 0.4243068 for 51 rounds
CV with max_depth=10, min_child_weight=6
	MAE 0.42898420000000004 for 64 rounds
CV with max_depth=10, min_child_weight=7
	MAE 0.43361279999999996 for 55 rounds
CV with max_depth=11, min_child_weight=5
	MAE 0.4243068 for 51 rounds
CV with max_depth=11, min_child_weight=6
	MAE 0.42898420000000004 for 64 rounds
CV with max_depth=11, min_child_weight=7
	MAE 0.43361279999999996 for 55 rounds
Best params: 9, 5, MAE: 0.4243068


In [22]:
params['max_depth'] = 9
params['min_child_weight'] = 5

### Parameters subsample and colsample_bytree

In [23]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [24]:
min_mae = float("Inf")
best_params = None

# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))

    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample

    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=101,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )

    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)

print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
	MAE 0.4243068 for 51 rounds
CV with subsample=1.0, colsample=0.9
	MAE 0.42271139999999996 for 55 rounds
CV with subsample=1.0, colsample=0.8


/Users/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


	MAE 0.42278860000000007 for 52 rounds
CV with subsample=1.0, colsample=0.7
	MAE 0.4246284 for 62 rounds
CV with subsample=0.9, colsample=1.0
	MAE 0.42861479999999996 for 22 rounds
CV with subsample=0.9, colsample=0.9
	MAE 0.4276216 for 22 rounds
CV with subsample=0.9, colsample=0.8
	MAE 0.4141916 for 43 rounds
CV with subsample=0.9, colsample=0.7
	MAE 0.42520600000000003 for 37 rounds
CV with subsample=0.8, colsample=1.0
	MAE 0.4281614 for 22 rounds
CV with subsample=0.8, colsample=0.9
	MAE 0.4294488 for 22 rounds
CV with subsample=0.8, colsample=0.8
	MAE 0.42231399999999997 for 47 rounds
CV with subsample=0.8, colsample=0.7
	MAE 0.4232164 for 47 rounds
CV with subsample=0.7, colsample=1.0
	MAE 0.4276708 for 47 rounds
CV with subsample=0.7, colsample=0.9
	MAE 0.4273924 for 36 rounds
CV with subsample=0.7, colsample=0.8
	MAE 0.4279984 for 36 rounds
CV with subsample=0.7, colsample=0.7
	MAE 0.4216202 for 47 rounds
Best params: 0.9, 0.8, MAE: 0.4141916


In [25]:
#again, update the parameters

params['subsample'] = 1.0
params['colsample_bytree'] = .8

### Parameter ETA

In [26]:
%time

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 12.2 µs


In [27]:
# This can take some time…
min_mae = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))

    # We update our parameters
    params['eta'] = eta

    # Run and time CV
    %time cv_results = xgb.cv (params,dtrain,num_boost_round=num_boost_round,seed=101,nfold=5,metrics=['mae'],early_stopping_rounds=10)

    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta

print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
CPU times: user 93.5 ms, sys: 11.5 ms, total: 105 ms
Wall time: 112 ms
	MAE 0.42278860000000007 for 52 rounds

CV with eta=0.2
CPU times: user 141 ms, sys: 17.1 ms, total: 158 ms
Wall time: 166 ms
	MAE 0.4220966 for 97 rounds

CV with eta=0.1


/Users/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  app.launch_new_instance()


CPU times: user 137 ms, sys: 15.5 ms, total: 153 ms
Wall time: 154 ms
	MAE 0.4278042 for 101 rounds

CV with eta=0.05
CPU times: user 446 ms, sys: 50.4 ms, total: 496 ms
Wall time: 499 ms
	MAE 0.42286979999999996 for 354 rounds

CV with eta=0.01
CPU times: user 1.16 s, sys: 130 ms, total: 1.29 s
Wall time: 1.3 s
	MAE 0.429995 for 996 rounds

CV with eta=0.005
CPU times: user 1.2 s, sys: 133 ms, total: 1.34 s
Wall time: 1.43 s
	MAE 0.4409797999999999 for 998 rounds

Best params: 0.2, MAE: 0.4220966


In [28]:
params['eta'] = .01

In [29]:
params

{'max_depth': 9,
 'min_child_weight': 5,
 'eta': 0.01,
 'subsample': 1.0,
 'colsample_bytree': 0.8,
 'objective': 'binary:logistic',
 'eval_metric': 'mae'}

#### Parameters
```
{'colsample_bytree': 0.8,
 'eta': 0.01,
 'eval_metric': 'mae',
 'max_depth': 9,
 'min_child_weight': 5,
 'objective': 'binary:logistic',
 'subsample': 1.0}
 ```

In [30]:
# Train a model with the new parameters, run on Test set
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[0]	Test-mae:0.499922
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:0.499845
[2]	Test-mae:0.499688
[3]	Test-mae:0.499227
[4]	Test-mae:0.49877
[5]	Test-mae:0.498317
[6]	Test-mae:0.49825
[7]	Test-mae:0.498183
[8]	Test-mae:0.497739
[9]	Test-mae:0.497299
[10]	Test-mae:0.497238
[11]	Test-mae:0.497175
[12]	Test-mae:0.496743
[13]	Test-mae:0.496314
[14]	Test-mae:0.49589
[15]	Test-mae:0.495147
[16]	Test-mae:0.49473
[17]	Test-mae:0.494598
[18]	Test-mae:0.494162
[19]	Test-mae:0.494031
[20]	Test-mae:0.493902
[21]	Test-mae:0.494087
[22]	Test-mae:0.493368
[23]	Test-mae:0.493089
[24]	Test-mae:0.492378
[25]	Test-mae:0.492236
[26]	Test-mae:0.492113
[27]	Test-mae:0.492062
[28]	Test-mae:0.492011
[29]	Test-mae:0.491642
[30]	Test-mae:0.491595
[31]	Test-mae:0.491476
[32]	Test-mae:0.491426
[33]	Test-mae:0.491021
[34]	Test-mae:0.49066
[35]	Test-mae:0.490261
[36]	Test-mae:0.489884
[37]	Test-mae:0.489772
[38]	Test-mae:0.489949
[39]	Test-mae:0.489904
[40]	Test-mae:0.490079
[41]	Test-mae:0.

Stopping. Best iteration:
```  
[315]	Test-mae:0.452347
```
Best MAE: 0.45 in 316 rounds

### MAE actually got *WORSE* !!

### Saving the model
Although we found the best number of rounds, our model has been trained with more rounds than optimal, thus before using it for predictions, we should retrain it with the good number of rounds. Since we now the exact best num_boost_round, we don't need the early_stopping_round anymore.

In [31]:
# 
num_boost_round = model.best_iteration + 1

best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-mae:0.499922
[1]	Test-mae:0.499845
[2]	Test-mae:0.499688
[3]	Test-mae:0.499227
[4]	Test-mae:0.49877
[5]	Test-mae:0.498317
[6]	Test-mae:0.49825
[7]	Test-mae:0.498183
[8]	Test-mae:0.497739
[9]	Test-mae:0.497299
[10]	Test-mae:0.497238
[11]	Test-mae:0.497175
[12]	Test-mae:0.496743
[13]	Test-mae:0.496314
[14]	Test-mae:0.49589
[15]	Test-mae:0.495147
[16]	Test-mae:0.49473
[17]	Test-mae:0.494598
[18]	Test-mae:0.494162
[19]	Test-mae:0.494031
[20]	Test-mae:0.493902
[21]	Test-mae:0.494087
[22]	Test-mae:0.493368
[23]	Test-mae:0.493089
[24]	Test-mae:0.492378
[25]	Test-mae:0.492236
[26]	Test-mae:0.492113
[27]	Test-mae:0.492062
[28]	Test-mae:0.492011
[29]	Test-mae:0.491642
[30]	Test-mae:0.491595
[31]	Test-mae:0.491476
[32]	Test-mae:0.491426
[33]	Test-mae:0.491021
[34]	Test-mae:0.49066
[35]	Test-mae:0.490261
[36]	Test-mae:0.489884
[37]	Test-mae:0.489772
[38]	Test-mae:0.489949
[39]	Test-mae:0.489904
[40]	Test-mae:0.490079
[41]	Test-mae:0.490033
[42]	Test-mae:0.489683
[43]	Test-mae:0.489022
[44

Use the new model to make predictions. Use the test dataset and compute MAE with the scikit-learn function. We should obtain the same score as promised in the last round of training.

In [32]:
mean_absolute_error(best_model.predict(dtest), y_test)

0.4841202879729478

In [33]:
# If you want to re-use your model on new data in the future, it can be a good idea 
#to save it to a file, here is how you can do it with XGBoost:

#best_model.save_model("my_model.model")

#You can then load the model later with:
#loaded_model = xgb.Booster()
#loaded_model.load_model("my_model.model")

# And use it for predictions.
#loaded_model.predict(dtest)